# Demo 2: PostgreSQL

## Walk through the basics of PostgreSQL <br><li>Create a Student Database <li>Creating Tables <li>Inserting data into tables<li>Running a simple SQL query to validate the information. <li>Performing Joins <li>Performing SUM and Group By statement.

#### Import the library 
Note: An error might popup after this command has executed. Read it carefully before proceeding.

In [ ]:
import psycopg2

### Create a connection to the database
1. Connect to the local instance of PostgreSQL (*127.0.0.1*)


#### Note: Adding the try except will make sure errors are caught and understood

In [ ]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 user=postgres password=12345678")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

### Use the connection to get a cursor that can be used to execute queries.

In [ ]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

### Use automatic commit so that each action is commited without having to call conn.commit() after each command. The ability to rollback and commit transactions is a feature of Relational Databases. 

In [ ]:
conn.set_session(autocommit=True)

### Test the Connection and Error Handling Code
The try-except block should handle the error: We are trying to do a select * on a table but the table has not been created yet.

In [ ]:
try: 
    cur.execute("select * from demodb.demotable")
except psycopg2.Error as e:
    print(e)

relation "demodb.demotable" does not exist
LINE 1: select * from demodb.demotable
                      ^



## Higher Education Commision Student's databse
<img src="sources/students.png">
<img src="sources/Universites.png">
<img src="sources/courses.png">


### Create a database to work in 

In [ ]:
try: 
    cur.execute("create database students")
except psycopg2.Error as e:
    print(e)

### Close our connection to the default database, reconnect to the students database, and get a new cursor.

In [ ]:
try: 
    conn.close()
except psycopg2.Error as e:
    print(e)

In [ ]:
try: 
    conn = psycopg2.connect("host=127.0.0.1 dbname=students user=postgres password=12345678")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)

In [ ]:
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

conn.set_session(autocommit=True)

### We will create our first table which contain information about Students. The students_info table has following columns. 
`column 1: Student_ID
 column 2: Name 
 column 3: University
 column 4: Year_of_admission
 column 5: Payable_amount
`

<img src="sources\students.png" width=500 height=250>

### Translate this information into a Create Table Statement. 

Review this document on PostgreSQL datatypes: https://www.postgresql.org/docs/9.5/datatype.html


In [ ]:
query = "CREATE TABLE IF NOT EXISTS student_info (student_id varchar, \
                                                  student_name varchar, \
                                                  university_id int, \
                                                  year_of_admission int, \
                                                  payable_amount int);"
try: 
    cur.execute(query)
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

### No error was found, but lets check to ensure our table was created.  `select count(*)` which should return 0 as no rows have been inserted in the table.

In [ ]:
try: 
    cur.execute("select count(*) from student_info")
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)
    
print(cur.fetchall())

[(0,)]


### Insert two rows 

In [ ]:
query = "INSERT INTO student_info (student_id, student_name, university_id, year_of_admission, payable_amount) \
                 VALUES (%s, %s, %s, %s, %s)"
try:
    cur.execute(query,("CS1701","Haseeb",1,2017,96000))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)
    
try: 
    cur.execute(query,("BS1902","Kashif",2,2019,14522))
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

### Validate your data was inserted into the table. 
The while loop is used for printing the results. If executing queries in the Postgres shell, this would not be required.

### Note: If you run the insert statement code more than once, you will see duplicates of your data. PostgreSQL allows for duplicates.

In [ ]:
try: 
    cur.execute("SELECT * FROM student_info;")
except psycopg2.Error as e: 
    print("Error: select *")
    print (e)

row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('CS1701', 'Haseeb', 1, 2017, 96000)
('BS1902', 'Kashif', 2, 2019, 14522)


### Insert Information using "execute many" statement

In [ ]:
data = [("BS1113","Rashid",1,2011,89000),
        ("CS1704","Hira",3,2017,102500),
        ("BS1105","Faisal",1,2011,36000)]

try:
    cur.executemany(query,data)
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

In [ ]:
cur.execute("SELECT * FROM student_info;")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('CS1701', 'Haseeb', 1, 2017, 96000)
('BS1902', 'Kashif', 2, 2019, 14522)
('BS1113', 'Rashid', 1, 2011, 89000)
('CS1704', 'Hira', 3, 2017, 102500)
('BS1105', 'Faisal', 1, 2011, 36000)


### Now we will create our second table which contain information about universities. The universities table has following columns. 
`column 1: university_id
 column 2: name 
`
<img src="sources\universites.png" width=500 height=250>

### Translate this information into a Create Table Statement. 

Review this document on PostgreSQL datatypes: https://www.postgresql.org/docs/9.5/datatype.html


In [ ]:
query = "CREATE TABLE IF NOT EXISTS universities (university_id int, university_name varchar);"
try: 
    cur.execute(query)
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

In [ ]:
data = [(1,"Information Technology University"),
        (2,"Baha Uddin Zakaria University"),
        (3,"Lahore University of Management Sciences")]

query = "INSERT INTO universities (university_id,university_name) VALUES (%s,%s)"

try:
    cur.executemany(query,data)
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

In [ ]:
cur.execute("SELECT * FROM universities;")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

(1, 'Information Technology University')
(2, 'Baha Uddin Zakaria University')
(3, 'Lahore University of Management Sciences')


### Lets create our third table which contain information about student's regitered courses. The courses table has following columns. 
`column 1: student_id
 column 2: subject_name 
`
<img src="sources\courses.png" width=500 height=250>

### Translate this information into a Create Table Statement. 

Review this document on PostgreSQL datatypes: https://www.postgresql.org/docs/9.5/datatype.html


In [ ]:
query = "CREATE TABLE IF NOT EXISTS courses (student_id varchar, subject_name varchar);"
try: 
    cur.execute(query)
except psycopg2.Error as e: 
    print("Error: Issue creating table")
    print (e)

In [ ]:
data = [("CS1701","Computer Architecture"),
("CS1701","Operating Systems"),
("CS1701","D Lab"),
("BS1902","Computing Fundamentals"),
("BS1902","Basic Electronics"),
("BS1113","Human Computer Interface"),
("CS1704","Computer Architecture"),
("CS1704","Operating Systems"),
("BS1105","Software Engineering")]

query = "INSERT INTO courses (student_id,subject_name) VALUES (%s,%s)"

try:
    cur.executemany(query,data)
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

In [ ]:
cur.execute("SELECT * FROM courses;")
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()

('CS1701', 'Computer Architecture')
('CS1701', 'Operating Systems')
('CS1701', 'D Lab')
('BS1902', 'Computing Fundamentals')
('BS1902', 'Basic Electronics')
('BS1113', 'Human Computer Interface')
('CS1704', 'Computer Architecture')
('CS1704', 'Operating Systems')
('BS1105', 'Software Engineering')


## Lets try using WHERE clause

In [ ]:
query = "SELECT * from student_info WHERE year_of_admission=2017"

In [ ]:
cur.execute(query)

rows = cur.fetchall()
for row in rows:
    print(row)

('CS1701', 'Haseeb', 1, 2017, 96000)
('CS1704', 'Hira', 3, 2017, 102500)


### Using the AND OR NOT operator

In [ ]:
query = "SELECT * from student_info WHERE year_of_admission=2017 AND university_id=1"

In [ ]:
cur.execute(query)

rows = cur.fetchall()
for row in rows:
    print(row)

('CS1701', 'Haseeb', 1, 2017, 96000)


In [ ]:
query = "SELECT * from student_info WHERE year_of_admission=2017 OR university_id=1"

In [ ]:
cur.execute(query)

rows = cur.fetchall()
for row in rows:
    print(row)

('CS1701', 'Haseeb', 1, 2017, 96000)
('BS1113', 'Rashid', 1, 2011, 89000)
('CS1704', 'Hira', 3, 2017, 102500)
('BS1105', 'Faisal', 1, 2011, 36000)


In [ ]:
query = "SELECT * from student_info WHERE NOT year_of_admission=2017"

In [ ]:
cur.execute(query)

rows = cur.fetchall()
for row in rows:
    print(row)

('BS1902', 'Kashif', 2, 2019, 14522)
('BS1113', 'Rashid', 1, 2011, 89000)
('BS1105', 'Faisal', 1, 2011, 36000)


## Using the LIKE operator
finding all students enrolled in bachelors using LIKE operator

In [ ]:
query = "SELECT * from student_info WHERE student_id like 'BS%%'"

In [ ]:
cur.execute(query)

rows = cur.fetchall()
for row in rows:
    print(row)

('BS1902', 'Kashif', 2, 2019, 14522)
('BS1113', 'Rashid', 1, 2011, 89000)
('BS1105', 'Faisal', 1, 2011, 36000)


## SELECT DISTINCT Examples
The following SQL statement selects only the DISTINCT values from the "Subject_name" column in the "courses" table:

In [ ]:
query = "SELECT DISTINCT subject_name from courses;"

In [ ]:
cur.execute(query)

rows = cur.fetchall()
for row in rows:
    print(row)

('Basic Electronics',)
('Computer Architecture',)
('Software Engineering',)
('Operating Systems',)
('Human Computer Interface',)
('D Lab',)
('Computing Fundamentals',)


## Using the COUNT statement to count distinct subjects

In [ ]:
query = "SELECT count(DISTINCT subject_name) from courses;"

In [ ]:
count = cur.execute(query)
count = cur.fetchall()
print(count)

[(7,)]


## Using the ORDER BY Key Word

In [ ]:
query = "select * from student_info ORDER BY payable_amount"

In [ ]:
cur.execute(query)

rows = cur.fetchall()
for row in rows:
    print(row)

('BS1902', 'Kashif', 2, 2019, 14522)
('BS1105', 'Faisal', 1, 2011, 36000)
('BS1113', 'Rashid', 1, 2011, 89000)
('CS1701', 'Haseeb', 1, 2017, 96000)
('CS1704', 'Hira', 3, 2017, 102500)


In [ ]:
query = "select * from student_info ORDER BY payable_amount DESC"

In [ ]:
cur.execute(query)

rows = cur.fetchall()
for row in rows:
    print(row)

('CS1704', 'Hira', 3, 2017, 102500)
('CS1701', 'Haseeb', 1, 2017, 96000)
('BS1113', 'Rashid', 1, 2011, 89000)
('BS1105', 'Faisal', 1, 2011, 36000)
('BS1902', 'Kashif', 2, 2019, 14522)


## Using the UPDATE statement

In [ ]:
query = "UPDATE student_info SET student_name = 'Javeria' where student_id = 'CS1704'"
cur.execute(query)

In [ ]:
query = "select * from student_info ORDER BY payable_amount DESC"
cur.execute(query)

rows = cur.fetchall()
for row in rows:
    print(row)

('CS1704', 'Javeria', 3, 2017, 102500)
('CS1701', 'Haseeb', 1, 2017, 96000)
('BS1113', 'Rashid', 1, 2011, 89000)
('BS1105', 'Faisal', 1, 2011, 36000)
('BS1902', 'Kashif', 2, 2019, 14522)


## Lets Perform JOINS

### Find out the courses each student is taking by JOINING student_info with courses 

In [ ]:
query = "select courses.student_id, student_name, subject_name from \
         student_info JOIN courses \
         ON student_info.student_id = courses.student_id;"

In [ ]:
cur.execute(query)

rows = cur.fetchall()
for row in rows:
    print(row)

('CS1701', 'Haseeb', 'Computer Architecture')
('CS1701', 'Haseeb', 'Operating Systems')
('CS1701', 'Haseeb', 'D Lab')
('BS1902', 'Kashif', 'Computing Fundamentals')
('BS1902', 'Kashif', 'Basic Electronics')
('BS1113', 'Rashid', 'Human Computer Interface')
('CS1704', 'Javeria', 'Computer Architecture')
('CS1704', 'Javeria', 'Operating Systems')
('BS1105', 'Faisal', 'Software Engineering')


### Lets find out which students are enrolled in 'Computer Architecture' subject

In [ ]:
query = "select courses.student_id, student_name, subject_name from \
         student_info JOIN courses \
         ON student_info.student_id = courses.student_id where subject_name='Computer Architecture'"

In [ ]:
cur.execute(query)

rows = cur.fetchall()
for row in rows:
    print(row)

('CS1701', 'Haseeb', 'Computer Architecture')
('CS1704', 'Javeria', 'Computer Architecture')


### Lets find out the list of students with their respective University name by performing JOIN of student_info table with universities table

In [ ]:
query = "select student_id, student_name, university_name from \
         student_info JOIN universities \
         ON student_info.university_id = universities.university_id"

In [ ]:
cur.execute(query)

rows = cur.fetchall()
for row in rows:
    print(row)

('CS1701', 'Haseeb', 'Information Technology University')
('BS1113', 'Rashid', 'Information Technology University')
('BS1105', 'Faisal', 'Information Technology University')
('BS1902', 'Kashif', 'Baha Uddin Zakaria University')
('CS1704', 'Javeria', 'Lahore University of Management Sciences')


### Drop the table to avoid duplicates and clean up

In [ ]:
try: 
    cur.execute("DROP table student_info")
    cur.execute("DROP table universities")
    cur.execute("DROP table courses")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)

###  Close the cursor and connection. 

In [ ]:
cur.close()
conn.close()